ANALISIS DE FRECUENCIAS CON TF-IDF

In [34]:
import pandas as pd
from pprint import pprint
df = pd.read_csv("pericias_medicas.csv", sep=",", encoding="utf-8")
df = df['text'].apply(str)

In [42]:
#aplicamos tecnicas de limpieza
import re
def limpiarTexto(text):
    text = re.sub(' +',' ', text)
    text = re.sub('\.\n+','.\n',text)
    text = re.sub(' +\n+','\n',text)
    text = re.sub('\n+','\n',text)
    text = re.sub('\d+\n','',text)
    text = re.sub('\x0c','',text)
    text.strip()
    return text

df1 = df.apply(str)
df1 = df.apply(limpiarTexto)



In [43]:
pprint(df1[0])

('\n'
 'Informa a continuación:\n'
 'J.T. PERITO MEDICO PRESENTA INFORME PERICIAL\n'
 'Autos: “BOIERO FRANCO IVAN c/SWISS MEDICAL ART S.A. s/Accidente - Ley '
 'Especial”\n'
 'Expediente N.º: 71399/I-PROEMIO:\n'
 'Sr. JUEZ NACIONAL:\n'
 'HORTAS MARIA ANDREA S. perito médico de oficio en los siguientes\n'
 'autos, con domicilio constituido en la calle Austria 1754 3º piso Dpto. 10 '
 'de esta\n'
 'Capital Z 172, tel. 4325-8802 domicilio electrónico mashortas@hotmail.com, '
 '27-\n'
 '16523394-3 en cumplimiento lo dispuesto por V.S. referente a los presentes '
 'autos\n'
 'y después de haber examinado a quién dijo ser Boiero Franco Iván quién se\n'
 'tomaron en el momento del examen el estado clínico actual se consideraron '
 'la\n'
 'capacidad genérica y en particular los puntos de pericia solicitados por las '
 'partes\n'
 'y/o V.S.:\n'
 'II-ANTECEDENTES DE INTERES MEDICO-LEGAL:\n'
 'Foja 3 Inicia la demanda\n'
 'Foja 6 vta. Hechos accidente in itinere refiere que el 20 de setiembre de

## Expresion regular para encontrar los titulos

In [44]:
titulos = (re.findall(r'\n.+[A-Z]:', df1[0]))
# inicio = df1[0].index(titulos[11])
# final = inicio + (len(titulos[11]))
# df1[0][inicio:final]
# posicionTitulos = titulos[11], inicio,final

In [45]:
#lista de titulos y ubicacion
titulosPosicion=[]
for i,j in enumerate(titulos):
    inicio = df1[0].index(titulos[i])
    final = inicio + (len(titulos[i]))
    palabrasPorTitulo = final-inicio
    titulosPosicion.append((titulos[i], inicio,final, palabrasPorTitulo))
pprint(titulosPosicion)

[('\nExpediente N.º: 71399/I-PROEMIO:', 149, 182, 33),
 ('\nSr. JUEZ NACIONAL:', 182, 201, 19),
 ('\nII-ANTECEDENTES DE INTERES MEDICO-LEGAL:', 754, 795, 41),
 ('\nIII-EXAMEN FISICO DEL ACTOR:', 2192, 2221, 29),
 ('\nIII-1 PROSOGRAFIA:', 2221, 2240, 19),
 ('\nIII-2 MOTIVO DE LA EVALUACION:', 2734, 2765, 31),
 ('\nIII-3 DATOS PERSONALES:', 2822, 2846, 24),
 ('\nIII-4 RELATO DE LOS HECHOS:', 3545, 3573, 28),
 ('\nIII-5-SEMIOLOGIA DE LAS AFECCIONES:', 5305, 5341, 36),
 ('\n2. TOBILLO:', 5751, 5763, 12),
 ('\n4. PSICOENTREVISTA:', 6520, 6540, 20),
 ('\n-EXAMENES COMPLEMENTARIOS:', 9632, 9659, 27),
 ('\nIV-CONSIDERACIONES MEDICO-LEGALES:', 9714, 9749, 35),
 ('\nVII-INCAPACIDAD:', 21569, 21586, 17),
 ('\n'
  'Incapacidad Parcial y Permanente del 7,02% según el Baremo Ley VIII '
  'CONTESTACION DE LOS PUNTOS DE PERICIA:',
  21829,
  21936,
  107),
 ('\nIX-PETITORIO:', 23163, 23177, 14)]
